https://github.com/fastai/courses/blob/master/deeplearning1/nbs/char-rnn.ipynb

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
import importlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import random
import time
import os
import pandas as pd
import csv
import math
import bcolz
import pickle
import re
import pathlib
import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

SOS_TOKEN = '<SOS>' # Start Of Word

# Data setup

In [3]:
# importing keras might cause problems with cudann version etc
# import keras # some good utils in here
# path = keras.utils.data_utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")

In [4]:
path = '/home/ohu/.keras/datasets/nietzsche.txt'
text = open(path).read().lower()
path
print('corpus length:', len(text))

'/home/ohu/.keras/datasets/nietzsche.txt'

corpus length: 600893


In [5]:
!tail {path} -n5

not to be regarded as a pure type. Knowledge and science--as far as they
existed--and superiority to the rest of mankind by logical discipline
and training of the intellectual powers were insisted upon by the
Buddhists as essential to sanctity, just as they were denounced by the
christian world as the indications of sinfulness.

In [6]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
''.join(chars)

total chars: 57


'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices['\n']

0

# Utils

In [8]:
def string_to_tensor(in_str, chars_index=char_indices, as_variable=True):
    """Onehot encoded tensor of string"""
    tensor_length = len(in_str)
    tensor = torch.zeros(1, tensor_length, len(chars_index))
    for li, letter in enumerate(in_str):
        tensor[0, li, chars_index[letter]] = 1
    if as_variable:
        tensor = Variable(tensor).cuda()
    return tensor

string_to_tensor('hello', as_variable=False).size()

torch.Size([1, 5, 57])

In [9]:
def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

# Char Rnn

In [290]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CharRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.layers = 1
        
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.lin_output = nn.Linear(hidden_size, input_size)
        
        
    def forward(self, input_char_vs, hidden = None):
        if hidden is None:
            hidden = self.init_hidden()
        
        rnn_outputs, hidden = self.rnn(input_char_vs, hidden)
        
        outputs = self.lin_output(rnn_outputs[0])
        outputs = F.log_softmax(outputs)

        return outputs, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.layers, 1, self.hidden_size)).cuda()
    
model = CharRNN(len(chars), 512).cuda()
model

CharRNN (
  (rnn): GRU(57, 512, batch_first=True)
  (lin_output): Linear (512 -> 57)
)

In [180]:
#tmp = Variable(torch.FloatTensor([2]).view(1,1,-1)).cuda()
#tmp = Variable(string_to_tensor('hello')).cuda()
tmp = model(string_to_tensor('hello'))
len(tmp[0][0])
tmp[0].size()
tmp[1].size()
#chars[tmp[0].topk(1)[1].data[0][0]]

57

torch.Size([5, 57])

torch.Size([1, 1, 512])

In [182]:
tmp[0][0].topk(1)[1].data[0]

24

# Eval

In [183]:
def print_example(iters=320, choice=True):
    seed_string="ethics is a basic foundation of all that"
    for i in range(iters):
        #x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        x = string_to_tensor(seed_string)
        output, hidden = model(x)
        output = output[-1]
        if choice:
            #next_char = np.random.choice(chars, p=F.softmax(output)[0].data.cpu().numpy())
            next_char = np.random.choice(chars, p=F.softmax(output).data.cpu().numpy())
        else:
            next_char_idx = output.topk(1)[1].data[0] # [0]
            next_char = chars[next_char_idx]
        # return next_char
        seed_string = seed_string + next_char
    print(seed_string)
#tmp = print_example(choice=False)
print_example(iters=40, choice=True)
print_example(iters=40, choice=False)

ethics is a basic foundation of all thatgwyj50q2c9b qgt656643
;cs9w(3?6(0ën]8f ,
ethics is a basic foundation of all that----------------------------------------


# Samples

In [184]:
def get_random_sample_string(length=41):
    sample_place = random.randint(0, len(text)-length-1)
    sample = text[sample_place:sample_place+length]
    return sample
get_random_sample_string()

', and mean to\nguard and protect it from e'

In [219]:
sample_size = 50000

sample_sentence_size = 40
sample_data = torch.zeros(sample_size, sample_sentence_size, len(chars)) # .cuda()
#sample_target = torch.zeros((sample_size, sample_sentence_size, 1), torch.LongTensor)
sample_target = torch.LongTensor(sample_size, sample_sentence_size).zero_() # .cuda()
for i in range(sample_size):
    sample = get_random_sample_string(sample_sentence_size+1)
    sample = [char_indices[c] for c in sample]
    for j in range(sample_sentence_size):
        sample_data[i][j][sample[j]] = 1
        sample_target[i][j] = sample[j+1]

In [228]:
i = 5
j = 10
sample_data[i].topk(1)[1][j][0] == sample_target[i][j-1]
''.join([chars[c[0]] for c in sample_data[i].topk(1)[1]])

True

'it was locke of whom schelling rightly s'

In [320]:
clip_parameters_value = 0.25

def train_single(optimizer, loss_function, use_teacher_forcing):
    model.zero_grad()
    loss = 0
    
    sample_i = random.randint(0, len(sample_data)-1)
    #x = sample_data[sample_i].view(1,40,len(chars))
    #y = sample_target[sample_i]
    
    x = sample_data[sample_i].view(1,40,len(chars))
    y = sample_target[sample_i]
    
    target = Variable(y).cuda()

    hidden = model.init_hidden()
    outputs, hidden = model(Variable(x).cuda(), hidden)
    
    loss += loss_function(outputs, target)

    if clip_parameters_value:
        torch.nn.utils.clip_grad_norm(model.parameters(), clip_parameters_value)
    
    loss.backward()
    optimizer.step()
    
    output_line = []
    for output in outputs:
        char_idx = output.topk(1)[1].data[0]
        output_line.append(chars[char_idx])

    return ''.join(output_line), loss.data[0]

tmp = train_single(torch.optim.Adam(model.parameters(), lr=0.0001), nn.NLLLoss(), use_teacher_forcing=False)
tmp

('                                        ', 3.9262466430664062)

In [230]:
tmp[0].size()
tmp[1].size()

torch.Size([40, 57])

torch.Size([40])

In [268]:
def train_iterations(n_iters=100000, lr=0.001, teacher_forcing_ratio=0,
                     print_every=10000, plot_every=1000):
    global optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    start = time.time()
    model.train()

    current_loss = 0
    current_loss_iter = 0
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        #use_teacher_forcing = random.random() < teacher_forcing_ratio
        use_teacher_forcing = False
        
        result, loss = train_single(optimizer=optimizer, loss_function=nn.NLLLoss(),
                                    use_teacher_forcing=use_teacher_forcing)
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            teacher_forcing_str = ""
            if use_teacher_forcing:
                teacher_forcing_str = "(forcing)"
            
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} {}".format(
                      model_training.iterations, iteration/n_iters, time_since(start),
                      current_loss/current_loss_iter, loss, result, teacher_forcing_str))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 0


In [254]:
from pytorch_utils_oh_3 import ModelTraining
MODEL_SAVE_PATH = 'char_rnn_fast_ai_testing_2'

In [269]:
model_training = ModelTraining(MODEL_SAVE_PATH, [model])

Save path: data/models/char_rnn_fast_ai_testing_2


In [270]:
train_iterations(n_iters=10, print_every=5)

     5  50% (   0m 0s)   3.957   |   3.86: s                                        
    10 100% (   0m 0s)   3.704   |   3.33: t                                        


In [271]:
train_iterations(n_iters=10000, print_every=1000)

  1010  10% (   0m 4s)   2.469   |   2.06:  ahrl     af thtl aonee uld  y  an the w 
  2010  20% (   0m 9s)   2.142   |   1.94:   elidtnd seaslct ohth ushthe mne of tot 
  3010  30% (  0m 13s)   1.979   |   2.03: n ng   ior txplpee  ahet  ahlh y afeahll 
  4010  40% (  0m 17s)   1.884   |   1.58: ireton ooctinn tn elf aiinett is tegeess 
  5010  50% (  0m 22s)   1.799   |   1.14: nttomething thet the statd nd tnd the so 
  6010  60% (  0m 26s)   1.738   |   2.09: hen  t10 ooosouedtoe hrrsose  tnd the pa 
  7010  70% (  0m 31s)   1.698   |   1.25: thet thes worne of tunf toutenpli of ton 
  8010  80% (  0m 36s)   1.653   |   1.81: h hn tot days torled bfrovieat  inl tfer 
  9010  90% (  0m 40s)   1.634   |   1.69:   oesl  and done tncasst ine anowl an ae 
 10010 100% (  0m 45s)   1.599   |   1.90: n irly tlsh d  and cictiuaid thy foperii 


In [272]:
print_example()

ethics is a basic foundation of all that they knowse!"

"40

=dect,s which has aid semptaticipal hos infucquinest, however
extradicarice as this want for this factly anwicchy from sugsifeclizes,
itsil
duponitions
of any tratth, tham only neighing which make to lece, ar way their
practed by this freeconical sciences if the osay, always
way is the retort of re


In [273]:
train_iterations(n_iters=10000, print_every=1000)

 11010  10% (   0m 4s)   1.577   |   1.50:   oh t seldly txot eealiog trilosophecal 
 12010  20% (   0m 8s)   1.558   |   1.57: h ootmonger df eree  
10. th hive aan in 
 13010  30% (  0m 13s)   1.551   |   1.68: i atnlhough  an an t praceatte on thmnh  
 14010  40% (  0m 17s)   1.520   |   1.67: e tt ioos totdot her dhat thes fawg ng   
 15010  50% (  0m 22s)   1.508   |   1.19:  stonning tfl fusht avrily tecardstimsel 
 16010  60% (  0m 26s)   1.499   |   1.30: hltr y tith ttcvtance,ond tilh tieedom o 
 17010  70% (  0m 31s)   1.486   |   1.32: af the srrn and txocarion ahas axpersed  
 18010  80% (  0m 35s)   1.500   |   1.43:  l al anserpourse whtl tonsesahe snpgena 
 19010  90% (  0m 40s)   1.460   |   1.61: ee ng oh tnher   wn ias alpeaan  tnd tlp 
 20010 100% (  0m 44s)   1.449   |   2.41:  rlng onei-theough aed aaein dionrs -the 


In [274]:
print_example()

ethics is a basic foundation of all that
will despritte. there is some onesfluness, consequently coula sees of wive id a than he who eaviless one would partic lofty sttent one's basig we, and the belief that which he say and such an act far as under the word about the
more standardance of "geats, and thes succe! which were would be accustome theme the consti


In [275]:
train_iterations(n_iters=10000, print_every=5000)

 25010  50% (  0m 22s)   1.440   |   1.76: hdt d iite aseem  to  th   wnsonmanity t 
 30010 100% (  0m 46s)   1.403   |   1.22: aoearerto anr own iirtues  aot ae ome a  


In [276]:
print_example()

ethics is a basic foundation of all that all rato one is all that he is abruth!


1 110

been that there is ewhicatantimis and hadgenerto inspieses age, in a partically, and in a performous moralic and it is whenner, has value
of conduct has the liverdly, threed, and not be "ture," and beants" man's to guise" and shill, seems a string at
allated the powerful


In [277]:
train_iterations(n_iters=10000, print_every=5000)

 35010  50% (  0m 23s)   1.380   |   1.41: aart dasdoess arr eof tttlience on aesig 
 40010 100% (  0m 46s)   1.353   |   1.11: n o ae ahe diart and rtul if thesks  

1 


In [278]:
print_example()

ethics is a basic foundation of all that and inquired about a hindities, as
trouble, thas the pessame tual act in the law of their laid, in keep some genius, which namuated, first, to de uniteding as circumstancle, its delusales, despicabld. hitherto sociated some vivius of himself so near the fuith as "fututions"--this mind,
in it. i unand, in the last cutt


In [279]:
train_iterations(n_iters=10000, print_every=5000)

 45010  50% (  0m 22s)   1.338   |   1.74: nlesrua  and aon be d  time and tol nemm 
 50010 100% (  0m 44s)   1.318   |   1.01:  toffecient  an iauld bave teen t w tn t 


In [280]:
print_example()

ethics is a basic foundation of all that wese most yet agreea,
terrible attitations which these of all these scale, if the most business of the head altherner of life, in fact, the world over religion in
mytorous appirable than the "bid hence
feels prospesses oneself, has do not
science, with the soul of secret distress and deplictful and's falient, is clain


In [281]:
train_iterations(n_iters=50000, print_every=10000)

 60010  20% (  0m 46s)   1.307   |   1.69:  teoegsdam pwn sike,-irl oo dl ant;tust  
 70010  40% (  1m 32s)   1.289   |   1.62:  tn  lf aart dntavse there  tnmeml  trne 
 80010  60% (  2m 17s)   1.284   |   1.17:  ttfe ms aesposid to thkk ond siught and 
 90010  80% (   3m 1s)   1.274   |   1.28: ns ond snsis norkh ahece bo ng tom tven  
100010 100% (  3m 46s)   1.260   |   1.13:  ttimself ia dosdsttvery ooelings aveny  


In [282]:
print_example()

ethics is a basic foundation of all that wisfined
good nor hand, from show, that makes than ne virtue; for the reason, among impolencusance and the like origin, and therefore, the wisfy of nature
and our!" everywheres.--everythingmands conduce interpreted, prolistic, and unsertionally originated
understand understanding--or cratics,-and superiority and hardl


In [283]:
print_example()

ethics is a basic foundation of all that we
have been a remarnative, and enchaning and art fallited in tormely of france. the inclination of the deportmanity inso our
condictly as well an oldners, of sphinx-like
priparanisms.--but it is the genuine saint, he is prularieve,
already neither freedoms, certaintiap regarding
things, but understand overwhradies ar


In [285]:
train_iterations(n_iters=100000, print_every=10000)

110010  10% (  0m 44s)   1.265   |   1.59: ers ng y sorl aoth tis wrisicil serensan 
120010  20% (  1m 30s)   1.260   |   1.11:  ess of dsdelfishnborives ahth tod, lt i 
130010  30% (  2m 15s)   1.259   |   1.52: t    eur soul ahlh t seofocsess setdensi 
140010  40% (   3m 1s)   1.249   |   1.13:   and ahe potter oion te wsserpretsd in  
150010  50% (  3m 47s)   1.259   |   1.06: i tt tot teecehibed ty the wertrrs of th 
160010  60% (  4m 31s)   1.261   |   0.93: he dosel ty of the r pxgironment  the r  
170010  70% (  5m 15s)   1.255   |   1.74: hred tnl the  aave totherto oioes  tnstb 
180010  80% (  5m 58s)   1.256   |   1.30: ton il drwerf th tuercome tarhphysica  o 
190010  90% (  6m 42s)   1.269   |   1.32:  r tg tsen the sest snd selpest tave on  
200010 100% (  7m 26s)   1.284   |   1.10: athe mesinning  tn ttirificedthe mamrifi 


In [286]:
print_example()

ethics is a basic foundation of all that hypocrity art the feelings of much
and is--what as show the sedisating as perhaps after long agar most singly; and is that for the awe operates his "war,
much mirror, "de" his "latter," would yet to be levilesing the emergaus proper any, the most power: it has there is no health, to which we more evil." the mestly con


In [287]:
print_example()

ethics is a basic foundation of all that the
first still tipet and obvicable convalial haraffelly developed and friendly, to faist, ntilled on the "monstrous"--and "the" in the order of
habliaring and experienced them, most science, one, there
are mellow betrayed, arises what extent really the europeoning for mirds apartous case or that the sounce has made a


In [289]:
# seconds per iteration
(7*60+26)/100000

0.00446